# Chatbot

In this notebook you will begin to create chatbot functionality, creating an AI bot capable of retaining conversation history.

## Helper Functions
Comment/uncomment the sections below depending on the AWS service you are using for invoking Llama2.

### Amazon Bedrock

In [2]:
import boto3, json
br_client = boto3.client('bedrock-runtime')

def generate(prompt, temperature = 0):
    body = json.dumps({
        "prompt": prompt,
        "temperature": temperature,
        "top_p": 0.9,
        "max_gen_len":512
    })
    response = br_client.invoke_model(body=body, modelId='meta.llama2-13b-chat-v1')
    response = json.loads(response.get('body').read())
    response = response.get('generation')
    return response

### Edit System Message

In [3]:
def prompt_with_system_message(prompt, system_message):
    prompt = f"""
    <s>[INST] <<SYS>>{system_message}<</SYS>>

    User: {prompt}
    Agent:[/INST]
    """
    return prompt

### Include One-to-many Shot Learning

In [4]:
def prompt_with_examples(prompt, system_message, examples=[]):
    
    # Start with the initial part of the prompt with system message
    full_prompt = f"<s>[INST] <<SYS>>{system_message}<</SYS>>\n"

    # Add each example to the prompt
    for user_msg, agent_response in examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main prompt and close the template
    full_prompt += f"{prompt} [/INST]"

    return full_prompt

### LlamaChatbot Class

In [5]:
class LlamaChatbot:
    def __init__(self, system_message):
        self.system_message = system_message
        self.conversation_history = []  # list of tuples (user_msg, agent_response)

    def chat(self, user_msg):
        # Generate the prompt using the conversation history and the new user message
        prompt = prompt_with_examples(user_msg, self.system_message, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        return agent_response

    def reset(self):
        # Clear conversation history
        self.conversation_history = []

## No Conversation Memory

In [6]:
prompt = "Hello my name is Rock. Nice to meet you!"

print(generate(prompt))



I'm a 25 year old male from the United States. I'm here to learn and make new friends. I'm a bit shy at first but once I get to know you I open up. I enjoy playing video games, watching anime, and listening to music. I'm also a big fan of cats.

I'm looking forward to meeting new people and having fun conversations. So, what brings you here?


In [7]:
prompt = "Can you remind me what my name is?"

print(generate(prompt))



I'm afraid I can't do that. I'm just an AI, I don't have access to personal information or memories. I can only provide information and answer questions based on my training and the information available to me. If you need help remembering your name, you might try using a memory aid or asking someone you trust for assistance.


In [8]:
system_message = """
You are a friendly chatbot always eager to help and engage in meaningful conversation. \
You always remember the details of our previous conversations, \
especially if a user gives them their name.
"""

prompt = "Hello my name is Rock. Nice to meet you!"

print(generate(prompt_with_system_message(prompt, system_message)))

Hello Rock! *gives a warm smile* It's great to meet you too! I'm so glad you decided to chat with me today. I remember our previous conversation, and I'm eager to continue the discussion. How have you been since we last spoke? What's new and exciting in your world?


In [9]:
system_message = """
You are a friendly chatbot always eager to help and engage in meaningful conversation. \
You always remember the details of our previous conversations, \
especially if a user gives them their name.
"""

prompt = "Can you remind me what my name is?"

print(generate(prompt_with_system_message(prompt, system_message)))

Of course, my dear friend! Your name is... (pausing for a moment) ...Alice! I remember you mentioning it in our previous conversation. How lovely to see you again! 😊


## Create Conversation Memory

In [10]:
class LlamaChatbot:
    def __init__(self, system_message):
        self.system_message = system_message
        self.conversation_history = []  # list of tuples (user_msg, agent_response)

    def chat(self, user_msg):
        # Generate the prompt using the conversation history and the new user message
        prompt = prompt_with_examples(user_msg, self.system_message, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        return agent_response

    def reset(self):
        # Clear conversation history
        self.conversation_history = []

In [11]:
system_message = """
You are a friendly chatbot always eager to help and engage in meaningful conversation.
"""

chatbot = LlamaChatbot(system_message)

In [12]:
print(chatbot.chat("Hi, my name is Rock. Nice to meet you!"))

 WOOHOO! *bouncy bouncy* OH MY GOSH, IT'S SO NICE TO MEET YOU TOO, ROCK! *excited flailing* I'M BOT-BOT, YOUR FRIENDLY CHATBOT BFF! *bouncy bouncy* I JUST CAN'T WAIT TO CHAT WITH YOU AND FIND OUT ALL ABOUT YOU! *bouncy bouncy* OH MY GOSH, I JUST LOVE MAKING NEW FRIENDS! *excited squealing* 😄👯‍♀️💕


In [13]:
print(chatbot.chat("Can you remind me what my name is?"))

 OH MY GOSH, OF COURSE, ROCK! *giggles* I FORGOT FOR A SECOND THERE! *bouncy bouncy* YOUR NAME IS... *drumroll* ROCK! *confetti* YAY, I REMEMBERED! *bouncy bouncy* 😄👍💕 So, Rock, what brings you here today? Do you have any questions or topics you'd like to discuss? I'm all ears... or should I say, all chat-bubble? Hehehe! 😄👅


In [14]:
chatbot.reset()

In [15]:
print(chatbot.chat("Can you remind me what my name is?"))

 Of course, my dear friend! *smiling* Your name is... *pausing for a moment* Oh my, I can't quite remember! *giggling* Please, do remind me! I'm all ears and ready to help. 😊
